# Run QASVM on IBMQ devices (montreal, toronto)

In [1]:
import matplotlib.pyplot as plt
import numpy as np

# TODO: choose config
I_HAVE_ACCESS = True # True
DATA_TYPE = 'balanced' # 'unbalanced'
DEVICE = 'montreal' # 'toronto'
TEST_SIZE = 4
MAXITER = 5
LAST_AVG = 2**4
DIR_NAME = 'ibmq_device_run_results'

C=None
k=1

In [2]:
from qiskit import IBMQ

In [3]:
import qiskit
qiskit.__qiskit_version__

{'qiskit-terra': '0.17.2', 'qiskit-aer': '0.8.2', 'qiskit-ignis': '0.6.0', 'qiskit-ibmq-provider': '0.12.3', 'qiskit-aqua': '0.9.1', 'qiskit': '0.25.4', 'qiskit-nature': None, 'qiskit-finance': None, 'qiskit-optimization': None, 'qiskit-machine-learning': None}

In [4]:
IBMQ.stored_account()
IBMQ.load_account()

KeyboardInterrupt: 

In [ ]:
# for SVM
from classifiers.convex.svm import BinarySVM
from classifiers.kernel import Kernel
from classifiers.datasets.dataloader import Example_4x2
# for QASVM
from classifiers.quantum import Qasvm_Mapping_4x2
from classifiers.quantum.qasvm import QASVM
from classifiers.quantum.quantum_circuits import AnsatzCircuit9
from classifiers.optimizer import tSPSA
from qiskit.utils import QuantumInstance
from qiskit.ignis.mitigation import CompleteMeasFitter
from qiskit.circuit.library import RealAmplitudes, EfficientSU2, NLocal, TwoLocal
from qiskit.providers.aer import AerSimulator

# for logging & visualization
import logging
from classifiers.callback import CostParamStorage
from torch.utils.tensorboard import SummaryWriter
from classifiers.visualization import Plot_Data, Plot_Log_From_SPSA, Plot_SVM, compare_svm_and_qasvm
if I_HAVE_ACCESS:
    if IBMQ.active_account() is None:
        IBMQ.load_account()
    provider = IBMQ.get_provider(hub='ibm-q-skku', group='skku', project='saint') # TODO: your provider
    backend = provider.get_backend(f'ibmq_{DEVICE}')
else:
    import dill
    with open(f'./summary_ipynb/2021/06 June/03 Thu/naive({DEVICE})/ibmq_{DEVICE}', 'rb') as _f:
        backend = dill.load(_f)

backend

## How to pick `layout`

### Priority 1. Should be one of four

- (yi - i1 - i0 - xi - a - xj - j0 - j1 - yj) connection
- (i1 - i0 - xi - a - xj - j0 - j1) + (i0 - yi) + (j0 - yj) connection
- (yi - i0 - i1 - xi - a - xj - j1 - j0 - yj) connection
- (i0 - i1 - xi - a - xj - j1 - j0) + (i1 - yi) + (j1 - yj) connection

### Priority 2. Should minimize TwoQubit (gate) error of possible connections in layout

### Priority 3. Should minimize SingleQubit (gate) error of qubits in layoyt

### Priority 4. Overall error on `*i` register should be lower than error on `*j` register

In [ ]:
# e.g.
# layout = Qasvm_Mapping_4x2(backend, a=3, i0=8, i1=11, xi=5, yi=14, j0=1, j1=4, xj=2, yj=7)
# layout = Qasvm_Mapping_4x2(backend, a=16, i0=11, i1=8, xi=14, yi=5, j0=22, j1=25, xj=19, yj=24)
# layout = Qasvm_Mapping_4x2(backend, a=3, i0=9, i1=11, xi=5, yi=8, j0=0, j1=4, xj=2, yj=1)
# layout = Qasvm_Mapping_4x2(backend, a=13, i0=10, i1=7, xi=12, yi=6, j0=11, j1=8, xj=14, yj=9) # TODO: Choose layout

# layout for Montreal / Nov.3.2021
layout = Qasvm_Mapping_4x2(backend, a=16, i0=13, i1=12, xi=14, yi=10, j0=22, j1=25, xj=19, yj=26)

layout.plot('physical')

In [ ]:
import json
from pathlib import Path
DATA_DIR = Path.cwd()/DIR_NAME/f'{DATA_TYPE}'/f'{DEVICE}'
DATA_DIR.mkdir(exist_ok = True, parents=True)

with open(DATA_DIR / 'layout.json', 'w') as fp:
    json.dump(layout.json_layout, fp)

In [ ]:
%matplotlib inline
_EPS = 0.01
X, y = Example_4x2(DATA_TYPE=='balanced')()
ax = plt.axes(projection='3d')

from qiskit.visualization.bloch import Bloch
B = Bloch(axes=ax)

def s2c(x):
    bloch = [1, x[0], x[1]]
    r, theta, phi = bloch[0], bloch[1], bloch[2]
    bloch[0] = r * np.sin(theta) * np.cos(phi)
    bloch[1] = r * np.sin(theta) * np.sin(phi)
    bloch[2] = r * np.cos(theta)
    return bloch

def c2s(bloch):
    x = [np.arccos(bloch[2]), np.arctan(bloch[1]/bloch[0])]
    return x

vecs = np.array(list(map(s2c, X)))
for v in vecs[y==0]:
    B.add_annotation(v, text='A', color='r')
for v in vecs[y==1]:
    B.add_annotation(v, text='B', color='b')
# B.add_annotation(vecs.mean(axis=0), text='M', color='k')
vecA = vecs[y==0].mean(axis=0)
vecA = vecA/np.linalg.norm(vecA)
vecB = vecs[y==1].mean(axis=0)
vecB = vecB/np.linalg.norm(vecB)
vecP = (vecA-vecB)/np.linalg.norm(vecA-vecB)
vecQ = -(vecA-vecB)/np.linalg.norm(vecA-vecB)
#B.add_annotation(vecP, text='P', color='r')
#B.add_annotation(vecQ, text='Q', color='b')

def test_data_gen(n, vecP, vecQ):
    Xt = np.array([np.linspace(0, 2*np.pi, n+1), c2s(vecP)[1]*np.ones(n+1)]).T
    Xt = Xt[1:]
    yt = np.where(np.dot(np.array(list(map(s2c, Xt))), vecP-vecQ)<0, 1, 0)
    return Xt, yt

Xt, yt = test_data_gen(TEST_SIZE, vecP, vecQ)

vecs = np.array(list(map(s2c, Xt)))
for v in vecs[yt==0]:
    B.add_annotation(v, text='*', color='r')
for v in vecs[yt==1]:
    B.add_annotation(v, text='*', color='b')

B.render()

In [ ]:
var_form = RealAmplitudes(2, reps=1)
#var_form = AnsatzCircuit9(2, reps=1, rotational_block='rx', entangling_block='cz')
#var_form = TwoLocal(2, reps=5, rotation_blocks='rz', entanglement_blocks='cx', entanglement='linear')
#var_form = EfficientSU2(2, reps=1)
quantum_instance = QuantumInstance(backend, shots=2**13, seed_transpiler=100, optimization_level=2, initial_layout=layout, measurement_error_mitigation_cls=CompleteMeasFitter)
qasvm = QASVM(X, y, quantum_instance=quantum_instance, C=None, k=10, option='Bloch_sphere', var_form=var_form).dual
print(qasvm.second_order_circuit.depth())
print(qasvm.first_order_circuit.depth())
qasvm.second_order_circuit.draw(fold=-1, output='mpl', idle_wires=False)

In [ ]:
from tqdm.notebook import tqdm
def find_min_seed(quantum_instance):
    old_tv = 1000
    for i in tqdm(range(100)):
        old_seed = quantum_instance.compile_config['seed_transpiler']
        quantum_instance.compile_config['seed_transpiler'] = i
        qasvm = QASVM(X, y, quantum_instance=quantum_instance, C=None, k=10, option='Bloch_sphere', var_form=var_form).dual
        new_tv = qasvm.second_order_circuit.depth() + qasvm.first_order_circuit.depth() + np.abs(qasvm.second_order_circuit.depth() - qasvm.first_order_circuit.depth())**2
        if new_tv < old_tv:
            old_tv = new_tv
        else:
            quantum_instance.compile_config['seed_transpiler'] = old_seed
    return quantum_instance.compile_config['seed_transpiler']


In [ ]:
seed=find_min_seed(quantum_instance)
print(seed)

In [ ]:
from copy import deepcopy
real_qi =  QuantumInstance(backend, shots=2**13, seed_transpiler=seed, optimization_level=2, initial_layout=layout, measurement_error_mitigation_cls=CompleteMeasFitter)
sim_qi = QuantumInstance(AerSimulator.from_backend(backend), shots=2**13, seed_transpiler=seed, optimization_level=2, initial_layout=layout, measurement_error_mitigation_cls=CompleteMeasFitter)
exact_qi = deepcopy(sim_qi)
exact_qi.backend.set_options(noise_model=None)

In [ ]:
real_qasvm = QASVM(X, y, quantum_instance=real_qi, C=C, k=k, option='Bloch_sphere', var_form=var_form).dual
print(real_qasvm.second_order_circuit.depth())
print(real_qasvm.first_order_circuit.depth())
real_qasvm.second_order_circuit.draw(fold=-1, output='mpl', idle_wires=False)

In [ ]:
sim_qasvm = deepcopy(real_qasvm)
sim_qasvm._quantum_instance = sim_qi
exact_qasvm = deepcopy(real_qasvm)
exact_qasvm._quantum_instance = exact_qi

uniform_qasvm = QASVM(X, y, quantum_instance=exact_qi, C=C, k=k, option='Bloch_uniform', var_form=None).dual

In [ ]:
"""from tqdm.notebook import tqdm
from qiskit.providers.ibmq import IBMQ
IBMQ.load_account()
qasm_qi = IBMQ.get_provider(group='open').get_backend('ibmq_qasm_simulator')
real_qasvm._quantum_instance = QuantumInstance(qasm_qi, shots=2**13, seed_transpiler=seed, optimization_level=2, initial_layout=layout, measurement_error_mitigation_cls=CompleteMeasFitter)
# if this works, then any error in `ibmq_device_run.ipynb` is from qiskit itself.
if I_HAVE_ACCESS:
    random_parameters = np.random.rand(real_qasvm.num_parameters)
    print(real_qasvm.cost_fn(random_parameters))
    print(real_qasvm.f(Xt[:1]))"""

In [ ]:
from tqdm.notebook import tqdm

if I_HAVE_ACCESS:
    real_storage = CostParamStorage()
    optimizer = tSPSA(maxiter=MAXITER, blocking=True, last_avg=LAST_AVG, callback=real_storage)

    for epoch in tqdm(range(1, MAXITER + 1)):
        optimizer.step(real_qasvm.cost_fn, real_qasvm.parameters)
        if real_storage.num_accepted() > 2 * LAST_AVG and real_storage.last_cost_avg(2 * LAST_AVG, ignore_rejected=True) < real_storage.last_cost_avg(LAST_AVG, ignore_rejected=True):
            break

    real_qasvm.parameters = real_storage.last_avg(LAST_AVG, ignore_rejected=True)
    real_fvec = real_qasvm.f(Xt)
    
    real_qasvm.save(DATA_DIR / 'real_qasvm')
    real_storage.save(DATA_DIR / 'real_storage')
    np.save(DATA_DIR / 'real_fvec.npy', real_fvec)

In [ ]:
sim_storage = CostParamStorage()
optimizer = tSPSA(maxiter=MAXITER, blocking=True, last_avg=LAST_AVG, callback=sim_storage)

for epoch in tqdm(range(1, MAXITER + 1)):
    optimizer.step(sim_qasvm.cost_fn, sim_qasvm.parameters)
    if sim_storage.num_accepted() > 2 * LAST_AVG and sim_storage.last_cost_avg(2 * LAST_AVG, ignore_rejected=True) < sim_storage.last_cost_avg(LAST_AVG, ignore_rejected=True):
        break

    # if too slow,
    # if sim_storage.num_accepted()>=LAST_AVG and sim_storage.last_cost_std(LAST_AVG, ignore_rejected=True)<=optimizer.allowed_increase/2:
    #     break
    

sim_qasvm.parameters = sim_storage.last_avg(LAST_AVG, ignore_rejected=True)
sim_fvec = sim_qasvm.f(Xt)

sim_qasvm.save(DATA_DIR / 'sim_qasvm')
sim_storage.save(DATA_DIR / 'sim_storage')
np.save(DATA_DIR / 'sim_fvec.npy', sim_fvec)

In [ ]:
exact_storage = CostParamStorage()
optimizer = tSPSA(maxiter=MAXITER, blocking=True, last_avg=LAST_AVG, callback=exact_storage)

for epoch in tqdm(range(1, MAXITER + 1)):
    optimizer.step(exact_qasvm.cost_fn, exact_qasvm.parameters)
    if exact_storage.num_accepted() > 2 * LAST_AVG and exact_storage.last_cost_avg(2 * LAST_AVG, ignore_rejected=True) < exact_storage.last_cost_avg(LAST_AVG, ignore_rejected=True):
        break

exact_qasvm.parameters = exact_storage.last_avg(LAST_AVG, ignore_rejected=True)
exact_fvec = exact_qasvm.f(Xt)

exact_qasvm.save(DATA_DIR / 'exact_qasvm')
exact_storage.save(DATA_DIR / 'exact_storage')
np.save(DATA_DIR / 'exact_fvec.npy', exact_fvec)

In [ ]:
uniform_fvec = uniform_qasvm.f(Xt)

uniform_qasvm.save(DATA_DIR / 'uniform_qasvm')
np.save(DATA_DIR / 'uniform_fvec.npy', uniform_fvec)

In [ ]:
svm = BinarySVM(Kernel('SingleQubit', reps=1), mutation='REDUCED_QASVM', C=C, k=k)
svm.fit(X, y)
Xtt, ytt = test_data_gen(100, vecP, vecQ)
svm_fvec = svm.f(Xtt)
svm.save(DATA_DIR / 'svm')
np.save(DATA_DIR / 'svm_fvec.npy', svm_fvec)

np.save(DATA_DIR / 'X.npy', X)
np.save(DATA_DIR / 'y.npy', y)
np.save(DATA_DIR / 'Xt.npy', Xt)
np.save(DATA_DIR / 'yt.npy', yt)
np.save(DATA_DIR / 'Xtt.npy', Xtt)
np.save(DATA_DIR / 'ytt.npy', ytt)

In [ ]:
# Unknown bug fix
if len(exact_fvec) != len(yt):
    exact_fvec = exact_fvec[4:]
if len(sim_fvec) != len(yt):
    sim_fvec = sim_fvec[4:]
if len(real_fvec) != len(yt):
    real_fvec = real_fvec[4:]
if len(uniform_fvec) != len(yt):
    uniform_fvec = uniform_fvec[4:]
if len(svm_fvec) != len(ytt):
    svm_fvec = svm_fvec[4:]

In [ ]:
plt.plot(Xtt[:, 0], svm_fvec, color='k', label='theory')
plt.scatter(Xt[yt==0, 0], exact_fvec[yt==0], c='r', marker='o', label='exact, A')
plt.scatter(Xt[yt==1, 0], exact_fvec[yt==1], c='b', marker='o', label='exact, B')
plt.scatter(Xt[yt==0, 0], sim_fvec[yt==0], c='r', marker='s', label='sim, A')
plt.scatter(Xt[yt==1, 0], sim_fvec[yt==1], c='b', marker='s', label='sim, B')
if I_HAVE_ACCESS:
    plt.scatter(Xt[yt==0, 0], real_fvec[yt==0], c='r', marker='d', label='real, A')
    plt.scatter(Xt[yt==1, 0], real_fvec[yt==1], c='b', marker='d', label='real, B')
plt.scatter(Xt[yt==0, 0], uniform_fvec[yt==0], c='r', marker='^', label='uniform, A')
plt.scatter(Xt[yt==1, 0], uniform_fvec[yt==1], c='b', marker='^', label='uniform, B')
plt.axhline(0, 0, 1, color='k', linestyle='--')
plt.legend()
plt.xlabel(r'$\theta$')
plt.title(DATA_TYPE)

In [ ]:
import qiskit.tools.jupyter
%qiskit_version_table
%qiskit_copyright